Hello  and welcome to Jon Dexter's ML class

In [1]:
#main libraries we using in the project
import pandas as pd
import json
from sklearn.model_selection import train_test_split        #for spliting our data into train & test
from sklearn.feature_extraction.text import CountVectorizer  # this lib actually
#converts data in text or images format to numerical formats for machine learnigng algorithms to be able to use
from sklearn import svm                                     #our classification lib

In [2]:
#our general classes

class Sentiments:
    NEGATIVE = 'NEGATIVE'
    POSITIVE = 'POSITIVE'
    NUETRAL = 'NEUTRAL'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiments.NEGATIVE
        elif self.score == 3:
            return Sentiments.NUETRAL
        else:
            return Sentiments.POSITIVE



In [3]:
#load data and store in a list... 
#note the list contains both the reviewText and its correspondind score, defined as the overall
reviews = []

with open('amazon_books_review_data_10000.json') as review_data:
    for line in review_data:
        line = json.loads(line)   ##########
        reviews.append(Review(line['reviewText'], line['overall']))   #adding to the list now...


In [4]:
#just trying to put the data unto a df
data = pd.DataFrame()

data['Text'] = [x.text for x in reviews]
data['Score'] = [x.score for x in reviews]
data['Sentiments'] = [x.get_sentiment() for x in reviews]

In [5]:
#we now split the data into train and test datasets 
#text is inputs, ie X and sentiments for output, ie Y

train, test = train_test_split(reviews, test_size=0.33, random_state=42)

In [6]:
train_input = [x.text for x in train]
train_output = [x.sentiment for x in train]

test_input = [y.text for y in test]
test_output = [y.sentiment for y in test]

Vecortize

In [7]:
#now we vectorise the training data.... note the trainning data is a text data, ml models wont be able to work on them well, so we have to find a way of converting them into numerical data... thats what we call the vectorization... find more info online for more clarification

vectorizer = CountVectorizer()    #instance of the method

vectorizer.fit(train_input)
train_input_vectors = vectorizer.transform(train_input)
#this two methods are standards, first you fit and then you transform

#now we have the actual data we will be using for the training ie...
#train_input_vectors
#train_output

#we need to also vectorize our input test datasets
test_input_vectors = vectorizer.transform(test_input)

SVM Model

In [8]:
#now we do some classification of the data
#there are several libraries for this but we going for SVC..... alert(l'll do a research to outline the various classification models available and thier pros and cons with some good recommendations )

svm_clf = svm.SVC(kernel='linear')                      #this is a linear classification

svm_clf.fit(train_input_vectors, train_output)          #pass the training input and output datasets

#then we try predicting something
svm_clf.predict(test_input_vectors[0])

#then we check the accuracy using this model


array(['POSITIVE'], dtype='<U8')

Decision tree

In [9]:
from sklearn.tree import DecisionTreeClassifier

dtc_clf = DecisionTreeClassifier()
dtc_clf.fit(train_input_vectors, train_output)

#predictiing somethins
dtc_clf.predict(test_input_vectors[0])

array(['POSITIVE'], dtype='<U8')

Naive Bayes

In [10]:
from sklearn.naive_bayes import GaussianNB

gnb_clf = GaussianNB()
gnb_clf.fit(train_input_vectors.toarray(), train_output)

#predicting
gnb_clf.predict(test_input_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(train_input_vectors, train_output)

#now we predict
lr_clf.predict(test_input_vectors[0])

/home/jondexter/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['POSITIVE'], dtype='<U8')

Evaluations

In [12]:
#Mean accuracy scores of all classification models used above

print(svm_clf.score(test_input_vectors, test_output))               #for SVM
print(dtc_clf.score(test_input_vectors, test_output))               #for Decison Tree
print(gnb_clf.score(test_input_vectors.toarray(), test_output))     #for Gausssian naive bayes
print(lr_clf.score(test_input_vectors, test_output))                #for Linear Regression

0.8124242424242424
0.7678787878787878
0.6587878787878788
0.8409090909090909


In [13]:
#F1 scores

from sklearn.metrics import f1_score

f1_score(test_output, svm_clf.predict(test_input_vectors), average=None, labels=[Sentiments.POSITIVE, Sentiments.NEGATIVE, Sentiments.NUETRAL])

#this computes the F1 score on all three unique sentiments in our output dataset ie the sentiment column
#however, the looking at the result, array([0.91319444, 0.22222222, 0.21052632])
#we realise that the predictions is skewed towards the positive sentiments,its highly likely to predict positive sentiment to the rest, the rest must also have a high f1 score not, not o.2 ....

array([0.90738061, 0.40268456, 0.2656    ])

So here's the fix... we need a larger dataset and we have to make sure the positive sentiments are at the same size as the negative.... just run a quick count and you seee the dataset has more positive sentiments than negative sentiments

In [68]:
#now lets do the magic
#lets have another class called ReviewContainer

import random

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    def DistributeEvenly(self):
        #our code below filters out the various sentiments and tries to evn them out
        negative = list(filter(lambda x: x.sentiment==Sentiments.NEGATIVE, reviews))
        positive = list(filter(lambda x: x.sentiment==Sentiments.POSITIVE, reviews))
        neutral  = list(filter(lambda x: x.sentiment==Sentiments.NUETRAL,  reviews))

        #we shrink the rest to the same size as the negative sentiments
        positive_shrink = positive[:len(negative)]
        neutral_shrink = neutral[:len(negative)]
        self.reviews = negative + positive_shrink + neutral_shrink

        random.shuffle(self.reviews)

        #print(len(positive_shrink))
        #print(len(negative))
        #print(len(neutral_shrink))
        #print(len(self.reviews))
        

        

In [69]:
#we take it on from the after we done with the spliting of the data
train_v2 = ReviewContainer(train)
test_v2 = ReviewContainer(test)
train_v2.DistributeEvenly()
train_v2_inputs = train_v2.get_text()
train_v2_outputs = train_v2.get_sentiment()

test_v2_inputs = test_v2.get_text()
test_v2_outputs = test_v2.get_sentiment()

In [70]:
print(train_v2_outputs.count(Sentiments.NEGATIVE))
print(train_v2_outputs.count(Sentiments.POSITIVE))
print(train_v2_outputs.count(Sentiments.NUETRAL))

644
644
644


In [72]:
#now that they are all of the same size, we can now vectorize and continue
vectorizer = CountVectorizer()    #instance of the method

vectorizer.fit(train_v2_inputs)
train_v2_inputs_vectors = vectorizer.transform(train_v2_inputs)

#now we have the actual data we will be using for the training ie...
#train_input_vectors
#train_output

#we need to also vectorize our input test datasets
test_v2_inputs_vectors = vectorizer.transform(test_v2_inputs)

In [75]:
#and now the svm model
svm_clf = svm.SVC(kernel='linear')                      #this is a linear classification

svm_clf.fit(train_v2_inputs_vectors, train_v2_outputs)          #pass the training input and output datasets

#then we try predicting something
svm_clf.predict(test_v2_inputs_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [77]:
f1_score(test_v2_outputs, svm_clf.predict(test_v2_inputs_vectors), average=None, labels=[Sentiments.POSITIVE, Sentiments.NEGATIVE, Sentiments.NUETRAL])
#well, it has increased small |(-^_^-)|

array([0.78659996, 0.4820394 , 0.4877193 ])

In [83]:
# and now some ramdom test 

test_data = ['I thorougly enjoyed this book, 5 star', 
            'Not bad', 
            'horrible waste of time', 
            'its a good book but do not buy okay', 
            'i will recommend it to no one']
transformed_test_data = vectorizer.transform(test_data)
svm_clf.predict(transformed_test_data)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE'],
      dtype='<U8')